<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/PrettyCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! pip install -q pyscipopt

In [7]:
import pandas as pd
from pyscipopt import Model, quicksum

In [8]:
! git clone https://github.com/Iremguel/Fallstudie_Elektrifizierung_der_Logistik.git

fatal: destination path 'Fallstudie_Elektrifizierung_der_Logistik' already exists and is not an empty directory.


In [9]:
folder = "Fallstudie_Elektrifizierung_der_Logistik/Daten/"

In [10]:
chargers = pd.read_csv(f"{folder}chargers.csv", sep=";")

In [11]:
diesel_trucks = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [12]:
electric_trucks = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [13]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [14]:
# =========================
# 2. Indexmengen
# =========================

T = routes["route_id"].tolist()

V_diesel = diesel_trucks["truck_model"].tolist()
V_electric = electric_trucks["truck_model"].tolist()
V = V_diesel + V_electric
E = V_electric

S = chargers["charger_model"].tolist()

K = range(96)  # 15-Minuten-Intervalle

In [15]:
distance = routes.set_index("route_id")["distance_total"].to_dict()
distance_toll = routes.set_index("route_id")["distance_toll"].to_dict()

In [16]:
fix_cost_diesel = (
    diesel_trucks["capex_yearly"]
    + diesel_trucks["opex_yearly"]
    + diesel_trucks["kfz_yearly"]
).to_dict()

fix_cost_diesel = dict(zip(diesel_trucks["truck_model"], fix_cost_diesel))

toll_cost = {v: 0.34 for v in V_diesel}


In [17]:
fix_cost_electric = (
    electric_trucks["capex_yearly"]
    + electric_trucks["opex_yearly"]
).to_dict()

fix_cost_electric = dict(zip(electric_trucks["truck_model"], fix_cost_electric))

energy_per_km = (
    electric_trucks["avg_energy_kWh_per_100km"] / 100
).to_dict()
energy_per_km = dict(zip(electric_trucks["truck_model"], energy_per_km))

max_charge_power_vehicle = dict(
    zip(electric_trucks["truck_model"], electric_trucks["max_power"])
)

battery_capacity = dict(
    zip(electric_trucks["truck_model"], electric_trucks["soc_max_kWh"])
)

thg_revenue = dict(
    zip(electric_trucks["truck_model"], electric_trucks["thg_yearly"])
)

for v in V_electric:
    toll_cost[v] = 0.0


In [18]:
charger_cost = (
    chargers["capex_yearly"] + chargers["opex_yearly"]
).to_dict()
charger_cost = dict(zip(chargers["charger_model"], charger_cost))

charger_power = dict(
    zip(chargers["charger_model"], chargers["max_power"])
)

charging_spots = dict(
    zip(chargers["charger_model"], chargers["charging_spots"])
)

MAX_CHARGERS = 3


In [19]:
ELECTRICITY_PRICE = 0.25
PEAK_PRICE = 150

GRID_BASE = 500
GRID_STEP = 500
GRID_STEP_COST = 10000

DAYS_PER_YEAR = 260
DT = 0.25


In [20]:
# =========================
# 4. Modell
# =========================

model = Model("E_LKW_Logistik")


In [21]:
# Fahrzeuge
x = {v: model.addVar(vtype="I", lb=0, name=f"x_{v}") for v in V}

# Ladesäulen
y = {s: model.addVar(vtype="I", lb=0, name=f"y_{s}") for s in S}

# Netzanschluss
z = model.addVar(vtype="I", lb=0, name="z_grid")

# Tourenzuordnung
u = {(t, v): model.addVar(vtype="B", name=f"u_{t}_{v}") for t in T for v in V}

# Ladeleistung
p = {(v, k): model.addVar(lb=0, name=f"p_{v}_{k}") for v in E for k in K}

# Netzleistung
P_grid = {k: model.addVar(lb=0, name=f"Pgrid_{k}") for k in K}
P_peak = model.addVar(lb=0, name="P_peak")


In [22]:
model.setObjective(

    # Fahrzeugkosten
    quicksum((fix_cost_diesel.get(v, 0) + fix_cost_electric.get(v, 0)) * x[v] for v in V)

    # Ladeinfrastruktur
    + quicksum(charger_cost[s] * y[s] for s in S)

    # Netzanschluss
    + GRID_STEP_COST * z

    # Maut
    + DAYS_PER_YEAR * quicksum(
        u[t, v] * distance_toll[t] * toll_cost[v]
        for t in T for v in V
    )

    # Stromkosten
    + DAYS_PER_YEAR * quicksum(P_grid[k] * DT * ELECTRICITY_PRICE for k in K)

    # Leistungspreis
    + PEAK_PRICE * P_peak

    # THG-Erlöse
    - quicksum(thg_revenue[v] * x[v] for v in E),

    sense="minimize"
)


In [23]:
# (1) Jede Tour genau einmal
for t in T:
    model.addCons(quicksum(u[t, v] for v in V) == 1)

# (2) Flottenverfügbarkeit
for v in V:
    model.addCons(quicksum(u[t, v] for t in T) <= x[v])

# (3) Energiebedarf e-Lkw
for v in E:
    model.addCons(
        quicksum(p[v, k] * DT for k in K)
        >= quicksum(u[t, v] * distance[t] * energy_per_km[v] for t in T)
    )

# (4) Ladeleistung Fahrzeug
for v in E:
    for k in K:
        model.addCons(p[v, k] <= max_charge_power_vehicle[v])

# (5) Ladeinfrastruktur
for k in K:
    model.addCons(
        quicksum(p[v, k] for v in E)
        <= quicksum(y[s] * charger_power[s] for s in S)
    )

# (6) Max. Anzahl Ladesäulen
model.addCons(quicksum(y[s] for s in S) <= MAX_CHARGERS)

# (7) Netzanschluss
for k in K:
    model.addCons(P_grid[k] <= GRID_BASE + GRID_STEP * z)

# (8) Peakdefinition
for k in K:
    model.addCons(P_grid[k] <= P_peak)


In [25]:
model.optimize()

In [27]:
status = model.getStatus()
print("Solver-Status:", status)

Solver-Status: inforunbd


In [28]:
if status == "optimal":
    sol = model.getBestSol()

    print("\n--- Optimale Lösung ---\n")

    print("Fahrzeuge:")
    for v in V:
        print(f"{v}: {model.getSolVal(sol, x[v]):.0f}")

    print("\nLadesäulen:")
    for s in S:
        print(f"{s}: {model.getSolVal(sol, y[s]):.0f}")

    print(f"\nNetzanschluss-Erweiterungen: {model.getSolVal(sol, z):.0f}")
    print(f"Jahres-Peakleistung: {model.getSolVal(sol, P_peak):.2f} kW")

    print("\nTourenzuordnung:")
    for t in T:
        for v in V:
            if model.getSolVal(sol, u[t, v]) > 0.5:
                print(f"{t} → {v}")

else:
    print("\n⚠️ Keine optimale Lösung gefunden.")
    print("Solver-Status:", status)



⚠️ Keine optimale Lösung gefunden.
Solver-Status: inforunbd
